# Initialization

In [11]:
# Google Earth Engine packages
import ee
import geemap

# other packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# GIS packages
from pysheds.grid import Grid
import fiona


#Define a function to plot the digital elevation model
def plotFigure(data, label, cmap='Blues'):
    plt.figure(figsize=(12,10))
    plt.imshow(data, extent=grid.extent)
    plt.colorbar(label=label)
    plt.grid()
    
    
ee_img = 'ee.image.Image'
ee_ico = 'ee.imagecollection.ImageCollection'

In [12]:
# initialize GEE at the beginning of session
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()         # authenticate when using GEE for the first time
    ee.Initialize()

In [3]:
# get config data
filename = 'output/dem_gee.tif'
output_folder = 'output/'

dem = [ee_img, 'CGIAR/SRTM90_V4','elevation','SRTM 90m']
#dem = [ee_img, 'USGS/SRTMGL1_003','elevation','SRTM NASA 30m']
#dem = [ee_img', 'MERIT/DEM/v1_0_3','dem', 'MERIT 30m']
#dem = [ee_ico, 'JAXA/ALOS/AW3D30/V3_2','DSM','ALOS DSM']

y, x = 42.300264, 78.091444

# Start GEE and find catchment area

In [4]:
# call map that centers aroung the region
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
if dem[0] == ee_img:
    image = ee.Image(dem[1])
elif dem[0] == ee_ico:
    image = ee.ImageCollection(dem[1]).select(dem[2]).mosaic()
    
srtm_vis = { 'bands': dem[2],
             'min': 0,
             'max': 6000,
            'palette': ['000000', '478FCD', '86C58E', 'AFC35E', '8F7131','B78D4F', 'E2B8A6', 'FFFFFF']
           }
Map.addLayer(image, srtm_vis, dem[3], True, 0.7)

In [ ]:
point = ee.Geometry.Point(x,y)
Map.addLayer(point,{'color': 'blue'},'Discharge Point');

box = point.buffer(30000).bounds()
Map.addLayer(box,{'color': 'grey'},'Catchment Area', True, 0.7);
Map.centerObject(box, zoom=9)

In [ ]:
for feature in Map.draw_features:
    f_type = feature.getInfo()['geometry']['type']
    if f_type == 'Point':
        point = feature.geometry()
        print("Manually set pouring point will be considered")
    elif f_type == 'Polygon':
        box = feature.geometry()
        print("Manually drawn box will be considered")

In [ ]:
geemap.ee_export_image(image, filename=filename, scale=30, region=box, file_per_band=False)

# Catchment deliniation

In [ ]:
DEM_file = filename
# Plot the DEM
grid = Grid.from_raster(DEM_file)
dem = grid.read_raster(DEM_file)
grid.view(dem)
# Fill depressions in DEM
flooded_dem = grid.fill_depressions(dem)
# Resolve flats in DEM
inflated_dem = grid.resolve_flats(flooded_dem)
# Specify directional mapping
#N    NE    E    SE    S    SW    W    NW
dirmap = (64, 128, 1, 2, 4, 8, 16, 32)
# Compute flow directions
fdir = grid.flowdir(inflated_dem, dirmap=dirmap)
#catch = grid.catchment(x=x, y=y, fdir=fdir, dirmap=dirmap, xytype='coordinate')
# Compute accumulation
acc = grid.accumulation(fdir)
# Snap pour point to high accumulation cell
x_snap, y_snap = grid.snap_to_mask(acc > 1000, (x, y))
# Delineate the catchment
catch = grid.catchment(x=x_snap, y=y_snap, fdir=fdir, xytype='coordinate')
# Clip the DEM to the catchment
grid.clip_to(catch)
clipped_catch = grid.view(catch)

In [ ]:
demView = grid.view(dem, nodata=np.nan)
plotFigure(demView,'Elevation')
plt.show()

In [ ]:
from shapely.geometry import Polygon
import pyproj
from shapely.geometry import shape
from shapely.ops import transform


## Create shapefile and save it
shapes = grid.polygonize()

schema = {
    'geometry': 'Polygon',
    'properties': {'LABEL': 'float:16'}
}

catchment_shape = {}
with fiona.open(output_folder+'catchment.shp', 'w',
                driver='ESRI Shapefile',
                crs=grid.crs.srs,
                schema=schema) as c:
    i = 0
    for shape, value in shapes:
        catchment_shape = shape
        rec = {}
        rec['geometry'] = shape
        rec['properties'] = {'LABEL' : str(value)}
        rec['id'] = str(i)
        c.write(rec)
        i += 1      

print(f"Mean catchment elevation is {str(np.nanmean(demView))} m")

In [ ]:
catchment = ee.Geometry.Polygon(catchment_shape['coordinates'])
Map.addLayer(catchment, {}, 'Catchment')

catchment_area = catchment.area().divide(1000*1000).getInfo()
print(f"Catchment area is {catchment_area} km²")

# Determine glaciers in catchment area

In [ ]:
"""
fc = ee.FeatureCollection('GLIMS/current').filterBounds(catchment)

Map.addLayer(fc, {}, "GLIMS")
Map.centerObject(fc, 3)
geemap.ee_to_shp(fc, filename=output_folder+'glims.shp', selectors=None)
"""

In [ ]:
"""
import geopandas as gpd
from urllib.request import urlopen
import json

glims = gpd.read_file(output_folder+'glims.shp')
glims_rgi = glims[glims['geog_area'].str.contains("Randolph Glacier Inventory")]

rgiids = []
for anlys_id in glims_rgi['anlys_id']:
    url = f'https://www.glims.org/services/glacierinfo_full?anlys_id={anlys_id}'
    response = urlopen(url)
    data_json = json.loads(response.read())
    if 'glac_info' in data_json:
        glac_info = data_json['glac_info']
        if 'rgiid' in glac_info:
            print(".", end = '')
            rgiids.append(glac_info['rgiid'])

print("\nDone!")
"""

In [5]:
import geopandas as gpd

catchment = gpd.read_file('output/catchment.shp')
rgi = gpd.read_file("zip://input/13_rgi60_CentralAsia.zip")

if rgi.crs != catchment.crs:
    print("CRS adjusted")
    rgi = rgi.to_crs(catchment.crs)

rgi_in_catchment = gpd.sjoin(rgi,catchment,how='inner',predicate='intersects')

CRS adjusted


In [6]:
fc = geemap.geopandas_to_ee(rgi_in_catchment)
Map.addLayer(fc, {}, "RGI60")

In [7]:
rgiids = rgi_in_catchment['RGIId'].sort_values()

areas = []
req_file_names = []
for rgiid in rgiids:
    req_file_names.append(f'{rgiid}_thickness.tif')
    area = rgiid.split('.')[0]
    if area not in areas: areas.append(area)

In [10]:
import requests
from zipfile import ZipFile
import io

cnt = 0
for area in areas:
    url = f'https://www.research-collection.ethz.ch/bitstream/handle/20.500.11850/315707/composite_thickness_RGI60-{area}.zip?sequence=15&isAllowed=y'

    r = requests.get(url, stream=True)

    with ZipFile(io.BytesIO(r.content), 'r') as zipObj:
        # Get a list of all archived file names from the zip
        listOfFileNames = zipObj.namelist()
        for fileName in listOfFileNames:
            file = fileName.split('/')[1]
            if file in req_file_names:
                cnt += 1
                zipObj.extract(fileName, output_folder+'RGI')
print(f'{cnt} files have been extracted')

52 files have been extracted
